In [10]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import numpy as np
from scipy import stats
import seaborn as sns
from functools import reduce
pd.options.mode.chained_assignment = None 
from statistics import mean
from Profile_Generation import *
from plotting import *
import warnings
warnings.simplefilter('ignore', np.RankWarning)
from scipy.optimize import curve_fit
import matplotlib.patches as patches
from scipy.optimize import curve_fit
from openpyxl import load_workbook
import math
from scipy.optimize import OptimizeWarning
warnings.simplefilter("ignore", OptimizeWarning)
import timeit
from sklearn.metrics import mean_squared_error
from matplotlib.pyplot import cm
from matplotlib.patches import Patch
from scipy import stats
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from mpl_toolkits.axes_grid1 import make_axes_locatable
from scipy.stats import levene
import sys

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
whole_day_original = pd.read_excel('CURATE_results.xlsx', sheet_name='result')
evening_original = pd.read_excel('CURATE_results_evening_dose.xlsx', sheet_name='result')

In [43]:
whole_day = whole_day_original.copy()
evening = evening_original.copy()

# Check if equal means in deviation: result, p = 0.75, equal
whole_day_deviation = whole_day[whole_day.method=="L_RW_wo_origin"].deviation.reset_index(name='deviation')[['deviation']]
evening_deviation = evening[evening.method=="L_RW_wo_origin"].deviation.reset_index(name='deviation')[['deviation']]

print(f'deviation p-value: {stats.ttest_rel(whole_day_deviation.deviation, evening_deviation.deviation).pvalue:.2f}')
print(f"deviation mean, whole_day = {whole_day_deviation.describe().loc['mean'][0]:.2f}")
print(f"deviation mean, evening = {evening_deviation.describe().loc['mean'][0]:.2f}\n")

# Check if equal means in abs_deviation: result p = 0.86, equal
whole_day_abs_deviation = whole_day[whole_day.method=="L_RW_wo_origin"].abs_deviation.reset_index(name='abs_deviation')[['abs_deviation']]
evening_abs_deviation = evening[evening.method=="L_RW_wo_origin"].abs_deviation.reset_index(name='abs_deviation')[['abs_deviation']]

print(f'abs deviation p-value: {stats.ttest_rel(whole_day_abs_deviation.abs_deviation, evening_abs_deviation.abs_deviation).pvalue:.2f}')
print(f"abs deviation mean, whole_day = {whole_day_abs_deviation.describe().loc['mean'][0]:.2f}")
print(f"abs deviation mean, evening = {evening_abs_deviation.describe().loc['mean'][0]:.2f}")
# print(f'whole_day:\n{whole_day[whole_day.method=="L_RW_wo_origin"].deviation.describe()}')
# print(f'evening:\n{evening[evening.method=="L_RW_wo_origin"].deviation.describe()}')

deviation p-value: 0.75
deviation mean, whole_day = 0.21
deviation mean, evening = 0.14

abs deviation p-value: 0.86
abs deviation mean, whole_day = 2.16
abs deviation mean, evening = 2.14


In [6]:
%%time
# ~5mins

execute_CURATE()

Patient #117 has insufficient/<3 predictions (1 predictions) (for linear)!
Patient #117 has insufficient/<3 predictions (0 predictions) (for quadratic)!
Patient #121 has insufficient unique dose-response pairs for calibration (for quad)!
Patient #126 has insufficient unique dose-response pairs for calibration (for quad)!
Patient #130 has insufficient/<3 predictions (2 predictions) (for linear)!
Patient #130 has insufficient/<3 predictions (1 predictions) (for quadratic)!
Patient #133 has insufficient/<3 predictions (0 predictions) (for linear)!
Patient #133 has insufficient unique dose-response pairs for calibration (for quad)!
Patients to exclude for linear methods: ['117', '130', '133']
Patients to exclude for quad methods: ['117', '121', '126', '130', '133']
Wall time: 10min 6s


In [ ]:
%%time
# Perform CV
five_fold_cross_val_results, five_fold_cross_val_results_summary = find_pop_tau_with_CV()
execute_CURATE_and_update_pop_tau_results('CV', five_fold_cross_val_results_summary, five_fold_cross_val_results)

# Perform LOOCV
five_fold_cross_val_results, five_fold_cross_val_results_summary = find_pop_tau_with_LOOCV()
execute_CURATE_and_update_pop_tau_results('LOOCV', five_fold_cross_val_results_summary, five_fold_cross_val_results)